# 할리스 매장정보 웹스크래핑 실습

In [1]:
# 1. 필요한 라이브러리 설치 및 임포트
!pip install requests beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 20.4 MB/s eta 0:00:00 0:00:01


## 2. 웹 페이지 요청하기
- 웹사이트에서 데이터를 가져오기

In [2]:


import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from typing import List, Dict

def get_hollys_store_info(page: int) -> str:
    """
    할리스 매장 정보 페이지의 HTML을 가져오는 함수
    
    Args:
        page (int): 페이지 번호
    Returns:
        str: 웹 페이지의 HTML 내용
    """
    url = "https://www.hollys.co.kr/store/korea/korStore2.do"
    params = {"pageNo": page}
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # 오류가 있으면 예외를 발생시킴
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"페이지 {page} 요청 중 에러 발생: {e}")
        return ""

# 테스트: 첫 페이지 가져오기
html = get_hollys_store_info(1)
print("HTML 일부:", html[:500])  # 처음 500자만 출력

HTML 일부: <!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">
<head>
	<title>할리스</title>
	<meta charset="UTF-8">

	<meta name="Subject" content="HOLLYS" />
	<meta name="Title" content="HOLLYS" />
	<meta name="Keywords" content="HOLLYS,HOLLYS,할리스,할리스,커피" />
	<meta name="Description" content="할리스는 1998년 국내 첫 에스프레소 커피전문점을 개점한 순수 국내브랜드로서 당당하게 시장 선점이라는 확고한 위치를 확보하고 국내 에스프레소 커피시장을 주도해 나가고 있습니다." />
	<meta name="Author" content="HOLLYS F&B" />
	<meta name="Publisher" content="HOLL


## 3. HTML 파싱하기
- BeautifulSoup를 사용하여 HTML에서 필요한 정보를 추출

In [4]:
def parse_store_info(html: str) -> List[Dict]:
    """
    HTML에서 매장 정보를 파싱하는 함수
    
    Args:
        html (str): 파싱할 HTML 문자열
    Returns:
        List[Dict]: 매장 정보 딕셔너리의 리스트
    """
    stores = []
    soup = BeautifulSoup(html, 'html.parser')
    
    tbody = soup.find("tbody")
    if not tbody:
        return stores
        
    for tr in tbody.find_all("tr"):
        tds = tr.find_all('td')
        if len(tds) < 6:
            continue
            
        store = {
            'region': tds[0].text.strip(),
            'name': tds[1].text.strip(),
            'status': tds[2].text.strip(),
            'address': tds[3].text.strip(),
            'service': ' '.join(img['alt'] for img in tds[4].find_all('img')),
            'phone': tds[5].text.strip()
        }
        stores.append(store)
        
    return stores

# 테스트: 첫 페이지 파싱하기
stores = parse_store_info(html)
print(f"첫 페이지에서 찾은 매장 수: {len(stores)}")
print("\n첫 번째 매장 정보:")
print(json.dumps(stores[0], ensure_ascii=False, indent=2))

첫 페이지에서 찾은 매장 수: 10

첫 번째 매장 정보:
{
  "region": "인천 남동구",
  "name": "간석오거리점2",
  "status": "영업중",
  "address": "인천광역시 남동구 남동대로 931 (간석동) 씨앤케이 웨딩홀",
  "service": "주차",
  "phone": "032-425-0915"
}


## 4. 데이터 저장하기
- 수집한 데이터를 CSV와 JSON 형식으로 저장

In [5]:
def save_to_files(stores: List[Dict], base_path: str = "./"):
    """
    매장 정보를 CSV와 JSON 파일로 저장하는 함수
    
    Args:
        stores (List[Dict]): 저장할 매장 정보 리스트
        base_path (str): 파일을 저장할 기본 경로
    """
    # CSV 파일로 저장
    df = pd.DataFrame(stores)
    csv_path = f"{base_path}hollys_stores.csv"
    df.to_csv(csv_path, encoding='utf-8', index=False)
    
    # JSON 파일로 저장
    json_path = f"{base_path}hollys_stores.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(stores, f, ensure_ascii=False, indent=4)
    
    print(f"CSV 파일 저장 완료: {csv_path}")
    print(f"JSON 파일 저장 완료: {json_path}")
    
    # 데이터 미리보기
    print("\n데이터 미리보기:")
    print(df.head())

# 테스트: 첫 페이지 데이터 저장하기
save_to_files(stores)

CSV 파일 저장 완료: ./hollys_stores.csv
JSON 파일 저장 완료: ./hollys_stores.json

데이터 미리보기:
       region      name status  \
0      인천 남동구   간석오거리점2    영업중   
1      대구 수성구  대구범어천로점2    영업중   
2      서울 강동구  길동포유르센티점    영업중   
3      서울 강동구   고덕비즈밸리점    영업중   
4  경남 창원시 의창구     창원팔용점    영업중   

                                             address      service  \
0                  인천광역시 남동구 남동대로 931 (간석동) 씨앤케이 웨딩홀           주차   
1  대구광역시 수성구 범어천로 200 (범어동, 범어월드메르디앙웨스턴카운티) 102호,...           주차   
2              서울특별시 강동구 진황도로 104 (길동) 1층 (101~102호)                
3          서울특별시 강동구 고덕비즈밸리로 38 (고덕동) KS타워 101호~102호       테라스 주차   
4          경상남도 창원시 의창구 창원대로18번길 6-8 (팔용동) 할리스 창원팔용점  테라스 흡연시설 주차   

          phone  
0  032-425-0915  
1  053-759-5779  
2   02-487-9997  
3             .  
4             .  


## 5. 전체 과정 실행하기

In [7]:
def main():
    stores = []
    
    # 매장 정보 수집
    print("할리스 매장 정보 수집 중...")
    for page in range(1, 50):  # 1~2 페이지만 수집 (테스트용)
        html = get_hollys_store_info(page)
        if html:
            page_stores = parse_store_info(html)
            stores.extend(page_stores)
            print(f"페이지 {page}: {len(page_stores)}개의 매장 정보 수집 완료")
    
    if not stores:
        print("매장 정보를 가져오는데 실패했습니다.")
        return
        
    print(f"\n총 {len(stores)}개의 매장 정보를 수집했습니다.")

    # 데이터 저장
    try:
        save_to_files(stores)
    except Exception as e:
        print(f"데이터 저장 중 에러 발생: {e}")

# 전체 과정 실행
main()

할리스 매장 정보 수집 중...
페이지 1: 10개의 매장 정보 수집 완료
페이지 2: 10개의 매장 정보 수집 완료
페이지 3: 10개의 매장 정보 수집 완료
페이지 4: 10개의 매장 정보 수집 완료
페이지 5: 10개의 매장 정보 수집 완료
페이지 6: 10개의 매장 정보 수집 완료
페이지 7: 10개의 매장 정보 수집 완료
페이지 8: 10개의 매장 정보 수집 완료
페이지 9: 10개의 매장 정보 수집 완료
페이지 10: 10개의 매장 정보 수집 완료
페이지 11: 10개의 매장 정보 수집 완료
페이지 12: 10개의 매장 정보 수집 완료
페이지 13: 10개의 매장 정보 수집 완료
페이지 14: 10개의 매장 정보 수집 완료
페이지 15: 10개의 매장 정보 수집 완료
페이지 16: 10개의 매장 정보 수집 완료
페이지 17: 10개의 매장 정보 수집 완료
페이지 18: 10개의 매장 정보 수집 완료
페이지 19: 10개의 매장 정보 수집 완료
페이지 20: 10개의 매장 정보 수집 완료
페이지 21: 10개의 매장 정보 수집 완료
페이지 22: 10개의 매장 정보 수집 완료
페이지 23: 10개의 매장 정보 수집 완료
페이지 24: 10개의 매장 정보 수집 완료
페이지 25: 10개의 매장 정보 수집 완료
페이지 26: 10개의 매장 정보 수집 완료
페이지 27: 10개의 매장 정보 수집 완료
페이지 28: 10개의 매장 정보 수집 완료
페이지 29: 10개의 매장 정보 수집 완료
페이지 30: 10개의 매장 정보 수집 완료
페이지 31: 10개의 매장 정보 수집 완료
페이지 32: 10개의 매장 정보 수집 완료
페이지 33: 10개의 매장 정보 수집 완료
페이지 34: 10개의 매장 정보 수집 완료
페이지 35: 10개의 매장 정보 수집 완료
페이지 36: 10개의 매장 정보 수집 완료
페이지 37: 10개의 매장 정보 수집 완료
페이지 38: 10개의 매장 정보 수집 완료
페이지 39: 10개의 매장 정보 수집 완료
페이지 40: 10개의 매장 